In [ ]:
from google.colab import files
uploaded = files.upload()

Saving amCharts (2).csv to amCharts (2).csv


In [ ]:
import pandas as pd
import statistics as stats
import numpy as np

pattern_range = 10
max_trend_points = 150
current_index = 0
x_axis = 'TEST_PERIOD_FORMATTED'
y_axis = 'ATT_TP_DL_MV'
trend_type = "TREND_TYPE"
file_name = 'amCharts (2).csv'

In [ ]:
data = pd.read_csv(file_name)
df = pd.DataFrame()
df[x_axis] = pd.to_datetime(data[x_axis])
df[y_axis] = data[y_axis]
df = df.dropna().reset_index(drop=True)
df[trend_type] = np.nan
df.head()

,TEST_PERIOD_FORMATTED,ATT_TP_DL_MV,TREND_TYPE
0,2019-07-02,38.88,NaN
1,2019-07-03,38.61,NaN
2,2019-07-04,38.41,NaN
3,2019-07-05,36.79,NaN
4,2019-07-06,38.89,NaN


In [ ]:
current_trend = [] # list of datapoints part of current trend

# for displaying on graph
anomalies = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
anomalies[x_axis] = df[x_axis]
shifts = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
shifts[x_axis] = df[x_axis]
trends = pd.DataFrame(index=range(df.shape[0]),columns=[x_axis, y_axis])
trends[x_axis] = df[x_axis]

# builds starting trend from pattern_range number of datapoints
for current_index in range(pattern_range):
  current_trend.append(df[y_axis][current_index])

In [ ]:
for current_index in range(pattern_range - 1, len(df[y_axis])):
  datapoint = df[y_axis][current_index]
  old_mean = stats.mean(current_trend)
  strdev = stats.stdev(current_trend, old_mean)

  # for trends data points on graph
  if (current_index + 1) % (pattern_range/2) == 0:
    trends.iat[current_index, 1] = old_mean
  
  # keeps the current trend list with current points, protects from over-saturation
  if len(current_trend) >= max_trend_points:
    current_trend.pop(0)

  # check if new datapoint is anomaly / trend (3 stdev out)
  if datapoint >= strdev * 3 + old_mean or datapoint <= old_mean - strdev * 3:
    temp_trend = []
    for i in range(pattern_range):
      if (current_index + i < len(df[y_axis])):
        temp_trend.append(df[y_axis][current_index + i])
    new_mean = stats.mean(temp_trend)

    if len(temp_trend) == pattern_range and (new_mean >= strdev * 3 + old_mean or new_mean <= old_mean - strdev * 3): # trend shift
      print("index: " + str(current_index) + " shift: " + str(df[x_axis][current_index]))
      if old_mean < new_mean:
        df[trend_type][current_index] = "shift up"
      elif old_mean > new_mean:
        df[trend_type][current_index] = "shift down"

      shifts.iat[current_index, 1] = datapoint
      current_trend.clear()
      current_trend.extend(temp_trend)
      current_index += pattern_range
    else: # anomaly
      print("index: " + str(current_index) + " anomaly: " + str(df[x_axis][current_index]))
      df[trend_type][current_index] = "anomaly"
      anomalies.iat[current_index, 1] = datapoint
  else:
    current_trend.append(datapoint)

index: 49 shift: 2019-08-20 00:00:00
index: 81 anomaly: 2019-09-21 00:00:00
index: 108 anomaly: 2019-10-18 00:00:00
index: 136 anomaly: 2019-11-18 00:00:00
index: 139 anomaly: 2019-11-21 00:00:00


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


index: 301 anomaly: 2020-05-02 00:00:00
index: 308 anomaly: 2020-05-09 00:00:00
index: 471 shift: 2020-10-22 00:00:00
index: 494 anomaly: 2020-11-14 00:00:00
index: 495 anomaly: 2020-11-15 00:00:00
index: 496 anomaly: 2020-11-16 00:00:00
index: 586 anomaly: 2021-02-15 00:00:00
index: 587 anomaly: 2021-02-16 00:00:00
index: 588 anomaly: 2021-02-17 00:00:00
index: 589 anomaly: 2021-02-18 00:00:00
index: 803 shift: 2021-09-25 00:00:00
index: 848 anomaly: 2021-11-09 00:00:00
index: 893 anomaly: 2021-12-24 00:00:00
index: 929 anomaly: 2022-01-29 00:00:00
index: 930 anomaly: 2022-01-30 00:00:00
index: 944 anomaly: 2022-02-13 00:00:00
index: 1007 anomaly: 2022-04-17 00:00:00
index: 1013 anomaly: 2022-04-23 00:00:00
index: 1014 anomaly: 2022-04-24 00:00:00
index: 1021 anomaly: 2022-05-01 00:00:00
index: 1063 anomaly: 2022-06-12 00:00:00


In [ ]:
def returnName(mean, first_mean, leniency):
  if mean + mean * leniency >= first_mean and mean - mean * leniency <= first_mean:
    return "consolidated"
  elif mean > first_mean:
    return "up"
  elif mean < first_mean:
    return "down"

# trend identification
first_mean = df[y_axis][0]
i = pattern_range/2 - 1
leniency = 0.001 # leniency for considering means to be equal, 0.1%
name = ""
while i + pattern_range/2 <= len(trends[y_axis]):
  mean = trends[y_axis][i]
  name = returnName(mean, first_mean, leniency)
  j = 0
  if not pd.isnull(mean):
    j = i - (pattern_range/2 - 1)
  else:
    i += pattern_range/2
    mean = trends[y_axis][i]
    name = returnName(mean, first_mean, leniency)
  while j <= i:
    val = df[trend_type][j]
    if pd.isnull(val):
      df[trend_type][j] = name
    j += 1

  first_mean = mean
  i += pattern_range/2

# trend identification for last values
if j < len(trends[y_axis]):
  mean = df[y_axis][len(df[y_axis]) - 1]
  name = returnName(mean, first_mean, leniency)
  while j < len(trends[y_axis]):
    val = df[trend_type][j]
    if pd.isnull(val):
      df[trend_type][j] = name
    j += 1

In [ ]:
# 179 for Jan 2020 consolidated data, 500 for Jan 2021 downards trend
scale = 0
for i in range(len(df[y_axis])):
  print(str(df[x_axis][i + scale]) + " type: " + str(df[trend_type][i + scale]))

2019-07-02 00:00:00 type: down
2019-07-03 00:00:00 type: down
2019-07-04 00:00:00 type: down
2019-07-05 00:00:00 type: down
2019-07-06 00:00:00 type: down
2019-07-07 00:00:00 type: down
2019-07-08 00:00:00 type: down
2019-07-09 00:00:00 type: down
2019-07-10 00:00:00 type: down
2019-07-11 00:00:00 type: down
2019-07-12 00:00:00 type: up
2019-07-13 00:00:00 type: up
2019-07-14 00:00:00 type: up
2019-07-15 00:00:00 type: up
2019-07-16 00:00:00 type: up
2019-07-17 00:00:00 type: down
2019-07-18 00:00:00 type: down
2019-07-19 00:00:00 type: down
2019-07-20 00:00:00 type: down
2019-07-21 00:00:00 type: down
2019-07-22 00:00:00 type: down
2019-07-23 00:00:00 type: down
2019-07-24 00:00:00 type: down
2019-07-25 00:00:00 type: down
2019-07-26 00:00:00 type: down
2019-07-27 00:00:00 type: up
2019-07-28 00:00:00 type: up
2019-07-29 00:00:00 type: up
2019-07-30 00:00:00 type: up
2019-07-31 00:00:00 type: up
2019-08-01 00:00:00 type: up
2019-08-02 00:00:00 type: up
2019-08-03 00:00:00 type: up
201

In [ ]:
from matplotlib.pylab import rcParams
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import math

fig = go.Figure()
fig.add_trace(go.Scatter(x=df.TEST_PERIOD_FORMATTED, y=df.ATT_TP_DL_MV, 
                         mode='lines', name=y_axis))

fig.add_trace(go.Scatter(x=anomalies.TEST_PERIOD_FORMATTED, y=anomalies.ATT_TP_DL_MV,
                         mode='markers', name='Anomaly [less than 10 points 3 sd above]'))

fig.add_trace(go.Scatter(x=shifts.TEST_PERIOD_FORMATTED, y=shifts.ATT_TP_DL_MV,
                         mode='markers', name='Shift [more than 9 points 3 sd above]'))

fig.add_trace(go.Scatter(x=trends.TEST_PERIOD_FORMATTED, y=trends.ATT_TP_DL_MV,
                         mode='markers', name='Trend [follows mean]'))

fig.update_layout(showlegend=True)
fig.show()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive

Mounted at /content/drive
/content/drive/My Drive


In [ ]:
mylist = list()
for i in range(len(trends.ATT_TP_DL_MV)):
  mylist.append(i)

In [ ]:
excel_df = pd.DataFrame(mylist, columns=['index'])

In [ ]:
import numpy as np

In [ ]:
for i in mylist:
  if np.isnan(shifts.ATT_TP_DL_MV[i]) == False:
    excel_df.loc[excel_df['index'] == i, 'value'] = shifts.ATT_TP_DL_MV[i]
    excel_df.loc[excel_df['index'] == i, 'type'] = 'shift'  
  elif np.isnan(anomalies.ATT_TP_DL_MV[i]) == False:
    excel_df.loc[excel_df['index'] == i, 'value'] = anomalies.ATT_TP_DL_MV[i]
    excel_df.loc[excel_df['index'] == i, 'type'] = 'anomalie' 
  elif np.isnan(trends.ATT_TP_DL_MV[i]) == False:
    excel_df.loc[excel_df['index'] == i, 'value'] = trends.ATT_TP_DL_MV[i]
    excel_df.loc[excel_df['index'] == i, 'type'] = 'trend'
  else:
    excel_df.loc[excel_df['index'] == i, 'value'] = None
    excel_df.loc[excel_df['index'] == i, 'type'] = None

In [ ]:
excel_df

,index,value,type
0,0,NaN,NaN
1,1,NaN,NaN
2,2,NaN,NaN
3,3,NaN,NaN
4,4,NaN,NaN
...,...,...,...
1104,1104,77.346333,trend
1105,1105,NaN,None
1106,1106,NaN,None
1107,1107,NaN,None


In [ ]:
null_ex_df = excel_df.dropna() 

In [ ]:
trends.ATT_TP_DL_MV.dropna()

9          38.393
14      38.555333
19        38.4575
24         38.416
29         38.571
          ...    
1084      76.5716
1089    76.740133
1094    76.942933
1099    77.141333
1104    77.346333
Name: ATT_TP_DL_MV, Length: 220, dtype: object

In [ ]:
shifts.ATT_TP_DL_MV.dropna()

49     41.16
471    57.97
803    74.28
Name: ATT_TP_DL_MV, dtype: object

In [ ]:
null_ex_df

,index,value,type
9,9,38.393000,trend
14,14,38.555333,trend
19,19,38.457500,trend
24,24,38.416000,trend
29,29,38.571000,trend
...,...,...,...
1084,1084,76.571600,trend
1089,1089,76.740133,trend
1094,1094,76.942933,trend
1099,1099,77.141333,trend


In [ ]:
shift = null_ex_df[null_ex_df['type'] == 'shift']
anomalie = null_ex_df[null_ex_df['type'] == 'anomalie']

shiftl = shift.iloc[-1]
anomaliel = anomalie.iloc[-1]

In [ ]:
if shiftl['index'] > anomaliel['index']:
  trend = null_ex_df[null_ex_df['index'] <= (shiftl['index'] - 1)]
  trendl = trend.iloc[-1]
  use_this = shiftl
else:
  trend = null_ex_df[null_ex_df['index'] <= (anomaliel['index'] - 1)]
  trendl = trend.iloc[-1]
  use_this = anomaliel

print(trendl)

index       1059
value    75.6186
type       trend
Name: 1059, dtype: object


In [ ]:
q = df.iloc[-1]
q

TEST_PERIOD_FORMATTED    2022-07-27 00:00:00
ATT_TP_DL_MV                           82.07
TREND_TYPE                                up
Name: 1108, dtype: object

In [ ]:
import openpyxl 
from openpyxl.styles import PatternFill

wb = openpyxl.load_workbook("output_intern.xlsx")
ws = wb['Sheet1']

fill_cell_green = PatternFill(patternType = 'solid', fgColor = '35FC03')
fill_cell_red = PatternFill(patternType = 'solid', fgColor = 'FC2C03')

ws['A2'] = 'ATT_TP_DL_MV'
ws['B1'] = 'National'

if ((len(excel_df) - 30) >= use_this['index']):
  ws['B2'].fill = fill_cell_red
  if use_this['type'] == 'shift':
    ws['B2'] = "S " + str(round((use_this['value'] - trendl['value']), 2))
  else:
    ws['B2'] = "A " + str(round((use_this['value'] - trendl['value']), 2))
else:
  ws['B2'].fill = fill_cell_green

wb.save("output_intern.xlsx")

In [ ]:
ws = wb['Sheet2']

for i in range(len(df[y_axis])):
  if df[trend_type][i + scale] != 'consalidated' or df[trend_type][i + scale] != 'up' or df[trend_type][i + scale] != 'down':
    ws['A'+str(i+1)] = df[x_axis][i + scale]
    ws['B'+str(i+1)] = df[trend_type][i + scale]

wb.save("output_intern.xlsx")